In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import time
import itertools
import click
import numpy as np
from scipy import stats

#https://stackoverflow.com/questions/8049520/web-scraping-javascript-page-with-python
from requests_html import HTMLSession, AsyncHTMLSession

# https://www.youtube.com/watch?v=U6gbGk5WPws&list=PLzMcBGfZo4-n40rB1XaJ0ak1bemvlqumQ&index=3&ab_channel=TechWithTim
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select

In [2]:
# https://stackoverflow.com/questions/37979644/parse-beautifulsoup-element-into-selenium
def xpath_soup(element):
    """
    Generate xpath of soup element
    :param element: bs4 text or node
    :return: xpath as string
    """
    components = []
    child = element if element.name else element.parent
    for parent in child.parents:
        """
        @type parent: bs4.element.Tag
        """
        previous = itertools.islice(parent.children, 0, parent.contents.index(child))
        xpath_tag = child.name
        xpath_index = sum(1 for i in previous if i.name == xpath_tag) + 1
        components.append(xpath_tag if xpath_index == 1 else '%s[%d]' % (xpath_tag, xpath_index))
        child = parent
    components.reverse()
    return '/%s' % '/'.join(components)

In [3]:
base_url = 'http://www.marathonguide.com/results/browse.cfm?MIDD=16010422'
years = [str(i) for i in range(2001, 2019)] # 2019 on are excluded due to covid / being virtual

for year in years:

    # go to starting webpage
    path = 'C:\Program Files (x86)\chromedriver.exe'
    driver = webdriver.Chrome(path)
    driver.get(base_url)
    soup = BeautifulSoup(driver.page_source)
    
    # links, to select year for search, click desired year
    year_select = driver.find_element(by=By.LINK_TEXT, value=year)
    year_select.click()
    
    # select first 100 results and go to result page    
    browse_start = Select(driver.find_element(by=By.NAME, value='RaceRange')).select_by_visible_text('1 - 100')
    view_button = driver.find_element(by=By.NAME, value='SubmitButton')
    view_button.click()
    
    # instantiate list for results (goes to df)  
    results = []
    
    # loop condition    
    entrants = int(driver.current_url.split('&')[-1].split('=')[-1])
    end = 100
    
    while end < entrants:
        try:

            # get current page html
            soup2 = BeautifulSoup(driver.page_source, 'lxml')
            rows = soup2.find_all('tr')
            tds = []
            for row in rows:
                tds.append(row.find_all('td'))


            entries = []
            for td in tds:
                if len(td) == stats.mode([len(td) for td in tds])[0][0]:
                    entries.append(td)

            for entry in entries:
                stripped_entries = []

                for data in entry:
                    stripped_entries.append(data.text.strip())

                results.append(stripped_entries)

            # go to next page of results
            soup2 = BeautifulSoup(driver.page_source, 'lxml')
            browse_more_tag = soup2.find('img', attrs={'alt': 'Later Runners'})
            xpath = xpath_soup(browse_more_tag)
            browse_more_element = driver.find_element(by=By.XPATH, value=xpath)
            browse_more_element.click()

            # small pause to load page    
            time.sleep(.25)   

            # update end to check loop condition            
            end = int(driver.current_url.split('&')[-2].split('=')[-1])
        
        except AttributeError:
            break

    # outside while loop, done getting data, make df, save as csv, go to next year
    soup2 = BeautifulSoup(driver.page_source, 'lxml')
    headers = soup2.find_all('th')
    columns = []

    for header in headers:
        columns.append(header.text.strip())

    df = pd.DataFrame(results, columns=columns)
    df.to_csv(f'../data/London_Data/Original/London_Results_{year}.csv', index=False)

    # give them a break 
    time.sleep(2)

C:\Users\David\AppData\Local\Temp/ipykernel_20960/2156114418.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)
